In [198]:
import numpy as np
import pandas as pd
import datetime

In [199]:
import warnings
warnings.filterwarnings('ignore')

In [200]:
# import data and transpose
filepath = 'D:\GeorgeLab\data\output_file_test.xlsx'
df_raw = pd.read_excel(filepath,header=None).T
df_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,4215,4216,4217,4218,4219,4220,4221,4222,4223,4224
0,Filename,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,...,Reward 200,Reward 201,Timeout Press 1,Timeout Press 2,Timeout Press 3,Timeout Press 4,Timeout Press 5,Timeout Press 6,Timeout Press 7,Timeout Press 8
1,C:\MED-PC IV\DATA\MTF134BC15HSOXYSHA01,2021-11-29,2021-11-29,F1501,0,0,1,15:14:20,17:41:30,GWAS 2hNEW,...,0,0,283,284,0,0,0,0,0,0
2,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1502,0,0,2,15:14:24,17:41:30,GWAS 2hNEW,...,0,0,0,0,0,0,0,0,0,0
3,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1503,0,0,3,15:14:27,17:41:30,GWAS 2hNEW,...,0,0,0,0,0,0,0,0,0,0
4,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1504,0,0,4,15:14:30,17:41:30,GWAS 2hNEW,...,0,0,5141,5145,5168,5172,0,0,0,0


In [201]:
# modify the header
new_header = df_raw.iloc[0]   #grab the first row for the header
df = df_raw[1:]               #take the data except the header row
df.columns = new_header 
df.reset_index(drop=True, inplace=True)
df.head()

,Filename,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,...,Reward 200,Reward 201,Timeout Press 1,Timeout Press 2,Timeout Press 3,Timeout Press 4,Timeout Press 5,Timeout Press 6,Timeout Press 7,Timeout Press 8
0,C:\MED-PC IV\DATA\MTF134BC15HSOXYSHA01,2021-11-29,2021-11-29,F1501,0,0,1,15:14:20,17:41:30,GWAS 2hNEW,...,0,0,283,284,0,0,0,0,0,0
1,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1502,0,0,2,15:14:24,17:41:30,GWAS 2hNEW,...,0,0,0,0,0,0,0,0,0,0
2,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1503,0,0,3,15:14:27,17:41:30,GWAS 2hNEW,...,0,0,0,0,0,0,0,0,0,0
3,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1504,0,0,4,15:14:30,17:41:30,GWAS 2hNEW,...,0,0,5141,5145,5168,5172,0,0,0,0
4,MTF134BC15HSOXYSHA01\n,2021-11-29,2021-11-29,F1505,0,0,5,15:14:46,17:41:30,GWAS 2hNEW,...,0,0,0,0,0,0,0,0,0,0


In [202]:
# add new column for visualization 
df['Start Datetime'] = pd.to_datetime(df['Start Date'] + ' ' + df['Start Time'])
df['End Datetime'] = pd.to_datetime(df['End Date'] + ' ' + df['End Time'])
df['Timeout'] = df['Active Lever Presses'] - df['Reward']

In [203]:
# drop unnecessary columns
df.drop(['Start Date','Start Time','End Date','End Time'], axis=1, inplace=True)

In [204]:
# change data types
cols = df.columns.tolist()
for col in cols:
    name = col.lower()
    if ('active' in name) or ('reward' in name) or ('timeout' in name):
        df[col] = df[col].astype('int32')
        df[col] = df[col].replace(0,np.nan)
    else:
        pass

In [205]:
# drop columns which all values are Nan
df.dropna(how='all', axis=1, inplace=True)

In [206]:
# reorder the columns
new_columns = df.columns.tolist()
new_columns.insert(0, new_columns.pop(new_columns.index('Subject')))
new_columns.insert(1, new_columns.pop(new_columns.index('Start Datetime')))
new_columns.insert(2, new_columns.pop(new_columns.index('End Datetime')))

idx = new_columns.index('Reward') + 1
new_columns.insert(idx, new_columns.pop(new_columns.index('Timeout')))

In [207]:
# fill the nan with 0
df = df[new_columns]
df.fillna(0,inplace=True)
df.head()

,Subject,Start Datetime,End Datetime,Filename,Experiment,Group,Box,MSN,FR,Active Lever Presses,...,Reward 15,Reward 16,Timeout Press 1,Timeout Press 2,Timeout Press 3,Timeout Press 4,Timeout Press 5,Timeout Press 6,Timeout Press 7,Timeout Press 8
0,F1501,2021-11-29 15:14:20,2021-11-29 17:41:30,C:\MED-PC IV\DATA\MTF134BC15HSOXYSHA01,0,0,1,GWAS 2hNEW,0,4.0,...,0.0,0.0,283.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0
1,F1502,2021-11-29 15:14:24,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,2,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F1503,2021-11-29 15:14:27,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,3,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F1504,2021-11-29 15:14:30,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,4,GWAS 2hNEW,0,10.0,...,0.0,0.0,5141.0,5145.0,5168.0,5172.0,0.0,0.0,0.0,0.0
4,F1505,2021-11-29 15:14:46,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,5,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [208]:
df.columns.tolist()

['Subject',
 'Start Datetime',
 'End Datetime',
 'Filename',
 'Experiment',
 'Group',
 'Box',
 'MSN',
 'FR',
 'Active Lever Presses',
 'Inactive Lever Presses',
 'Reward',
 'Timeout',
 'Active 1',
 'Active 2',
 'Active 3',
 'Active 4',
 'Active 5',
 'Active 6',
 'Active 7',
 'Active 8',
 'Active 9',
 'Active 10',
 'Active 11',
 'Active 12',
 'Active 13',
 'Active 14',
 'Active 15',
 'Active 16',
 'Active 17',
 'Active 18',
 'Active 19',
 'Active 20',
 'Active 21',
 'Active 22',
 'Active 23',
 'Active 24',
 'Inactive 1',
 'Inactive 2',
 'Inactive 3',
 'Inactive 4',
 'Inactive 5',
 'Inactive 6',
 'Inactive 7',
 'Reward 1',
 'Reward 2',
 'Reward 3',
 'Reward 4',
 'Reward 5',
 'Reward 6',
 'Reward 7',
 'Reward 8',
 'Reward 9',
 'Reward 10',
 'Reward 11',
 'Reward 12',
 'Reward 13',
 'Reward 14',
 'Reward 15',
 'Reward 16',
 'Timeout Press 1',
 'Timeout Press 2',
 'Timeout Press 3',
 'Timeout Press 4',
 'Timeout Press 5',
 'Timeout Press 6',
 'Timeout Press 7',
 'Timeout Press 8']

In [209]:
df.to_excel('output_clean.xlsx')

----------------------

In [210]:
# transforming time
df_time = df.copy()
cols = df_time.columns.tolist()
cutoff = cols.index('Active 1')

In [211]:
time_begin = df_time['Start Datetime'].min().time()
time_to_minus = datetime.timedelta(hours=time_begin.hour,
                                   minutes=time_begin.minute,
                                   seconds=time_begin.second)

In [212]:
def tranform_to_time(row):
    start_time = row['Start Datetime']
    for col in cols[cutoff:]:
        secs = row[col]
        if secs != 0:
            row[col] = start_time + datetime.timedelta(seconds=secs) - time_to_minus
        else:
            pass
    return row

In [213]:
df_time = df_time.apply(tranform_to_time,axis=1)
df_time.head()

,Subject,Start Datetime,End Datetime,Filename,Experiment,Group,Box,MSN,FR,Active Lever Presses,...,Reward 15,Reward 16,Timeout Press 1,Timeout Press 2,Timeout Press 3,Timeout Press 4,Timeout Press 5,Timeout Press 6,Timeout Press 7,Timeout Press 8
0,F1501,2021-11-29 15:14:20,2021-11-29 17:41:30,C:\MED-PC IV\DATA\MTF134BC15HSOXYSHA01,0,0,1,GWAS 2hNEW,0,4.0,...,0.0,0.0,2021-11-29 00:04:43,2021-11-29 00:04:44,0.0,0.0,0.0,0.0,0.0,0.0
1,F1502,2021-11-29 15:14:24,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,2,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F1503,2021-11-29 15:14:27,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,3,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F1504,2021-11-29 15:14:30,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,4,GWAS 2hNEW,0,10.0,...,0.0,0.0,2021-11-29 01:25:51,2021-11-29 01:25:55,2021-11-29 01:26:18,2021-11-29 01:26:22,0.0,0.0,0.0,0.0
4,F1505,2021-11-29 15:14:46,2021-11-29 17:41:30,MTF134BC15HSOXYSHA01\n,0,0,5,GWAS 2hNEW,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
# Number of zeros
df_time[cols[cutoff:]].replace(0,np.nan).isna().sum().sum()

732

In [228]:
stats = []
for i in range(16):
    test_row = df_time.iloc[i]
    for col in cols[cutoff:]:
        stat = []
        if type(test_row[col]) != float:
            print(test_row['Subject'],col, test_row[col])
            print('-----')
            stat.append(test_row['Subject'])
            stat.append(test_row[col])
            stat.append(test_row[col] + datetime.timedelta(seconds=20))
            stat.append(col.split(' ')[0])
            stats.append(stat)
        else:
            pass

F1501 Active 1 2021-11-29 00:00:26
-----
F1501 Active 2 2021-11-29 00:04:43
-----
F1501 Active 3 2021-11-29 00:04:43
-----
F1501 Active 4 2021-11-29 00:04:44
-----
F1501 Inactive 1 2021-11-29 00:20:50
-----
F1501 Reward 1 2021-11-29 00:00:26
-----
F1501 Reward 2 2021-11-29 00:04:43
-----
F1501 Timeout Press 1 2021-11-29 00:04:43
-----
F1501 Timeout Press 2 2021-11-29 00:04:44
-----
F1503 Inactive 1 2021-11-29 01:52:59
-----
F1504 Active 1 2021-11-29 00:01:47
-----
F1504 Active 2 2021-11-29 01:25:37
-----
F1504 Active 3 2021-11-29 01:25:51
-----
F1504 Active 4 2021-11-29 01:25:55
-----
F1504 Active 5 2021-11-29 01:26:16
-----
F1504 Active 6 2021-11-29 01:26:18
-----
F1504 Active 7 2021-11-29 01:26:22
-----
F1504 Active 8 2021-11-29 01:27:09
-----
F1504 Active 9 2021-11-29 01:27:31
-----
F1504 Active 10 2021-11-29 01:28:13
-----
F1504 Inactive 1 2021-11-29 01:26:25
-----
F1504 Inactive 2 2021-11-29 01:27:56
-----
F1504 Inactive 3 2021-11-29 01:27:57
-----
F1504 Inactive 4 2021-11-29 01:2

In [229]:
len(stats)

148

In [230]:
timeline_data = pd.DataFrame(stats,columns = ['Subject','Start','End','Type'])
timeline_data

,Subject,Start,End,Type
0,F1501,2021-11-29 00:00:26,2021-11-29 00:00:46,Active
1,F1501,2021-11-29 00:04:43,2021-11-29 00:05:03,Active
2,F1501,2021-11-29 00:04:43,2021-11-29 00:05:03,Active
3,F1501,2021-11-29 00:04:44,2021-11-29 00:05:04,Active
4,F1501,2021-11-29 00:20:50,2021-11-29 00:21:10,Inactive
...,...,...,...,...
143,F1516,2021-11-29 00:53:37,2021-11-29 00:53:57,Timeout
144,F1516,2021-11-29 00:53:39,2021-11-29 00:53:59,Timeout
145,F1516,2021-11-29 01:59:47,2021-11-29 02:00:07,Timeout
146,F1516,2021-11-29 02:17:54,2021-11-29 02:18:14,Timeout


In [235]:
timeline_data.to_excel('timeline_data.xlsx')

In [231]:
import plotly.express as px

In [232]:
def make_timeline(df):
    fig = px.timeline(df, x_start="Start", x_end="End", y="Subject",color='Type')
    
    fig.update_xaxes(
        tickformat="%H:%M:%S",
        tickformatstops=[
            dict(dtickrange=[1800000, 10800000], value="%H:%M")]  # range is 1 hour to 24 hours
    )
    
    fig.update_yaxes(categoryorder="category ascending")
    
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        })
    
    fig.show()

In [197]:
make_timeline(timeline_data[timeline_data['Type']=='Inactive'])

In [233]:
make_timeline(timeline_data)

In [234]:
make_timeline(timeline_data[timeline_data['Subject']=="F1516"])